# Evaluation of SRIM stopping power extrapolations to transuranium elements

In [ ]:
%run GenerateStoppingPower.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.linalg
from matplotlib import colors
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

color_list = plt.rcParams['axes.prop_cycle'].by_key()['color'];
#Customising style
#mpl.rc('lines', linewidth=2, color='r')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 24
#mpl.rcParams['savefig.format'] = "eps"
#mpl.rcParams['savefig.dpi'] = 100
#mpl.rcParams['text.usetex'] = True
#mpl.rcParams['figure.figsize'] = (16,9)

## Ion

Example SiO2 layer as material

In [ ]:
# Construct a layer of UO2, arbitrary width (as it is not relevant for these calcs)
layer = Layer({
    'Si': {
        'stoich': 1,
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    },
    'O': {
        'stoich': 2,
        #'E_d': 20.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=2.32, width=10000.0)

Example ion is No-254.

In [ ]:
# Construct ion
A, E_per_A_max = 254, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)

In [ ]:
U254 = GenerateStoppingPower(layer, ion)

In [ ]:
No254 = GenerateStoppingPower(layer, ion, z_ion=102)

In [ ]:
# Construct ion
A, E_per_A_max = 289, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)
Fl289 = GenerateStoppingPower(layer, ion, z_ion=114)

In [ ]:
plt.figure()
plt.plot(U254.energy_raw, U254.dEdX_raw, label='U254')
plt.plot(No254.energy_raw, No254.dEdX_raw, label='No254')
plt.plot(Fl289.energy_raw, Fl289.dEdX_raw, label='Fl289')
plt.xlabel('Energy (MeV)')
plt.ylabel('Stopping power (MeV/µm)')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(U254.energy_raw, U254.GetRange(U254.energy_raw), label='U254_raw')
plt.plot(U254.energy_raw, U254.GetRange(U254.energy_raw, from_sr=True), label='U254')
plt.plot(No254.energy_raw, No254.GetRange(No254.energy_raw, from_sr=True), label='No254')
plt.plot(Fl289.energy_interp, Fl289.GetRange(Fl289.energy_interp), label='Fl289_raw')
plt.plot(Fl289.energy_interp, Fl289.GetRange(Fl289.energy_interp, from_sr=True), label='Fl289')

plt.xlabel('Energy (MeV)')
plt.ylabel('Range (µm)')
plt.legend()
plt.show()

## Material

Example PuO2 layer as material.

Method goes as follows: 

1. First construct the layer, but replacing the transuranic element with uranium (this is denoted the reference layer). 
2. As usual, construct the ion.
3. Calculate mean/effective nuclear charge for the reference layer and the desired layer.
4. Generate the stopping power providing objects given in points 1-3.

1. Construct layer

In [ ]:
# Construct a layer of SiO2, arbitrary width (as it is not relevant for these calcs)
layer = Layer({
    'U': {
        'stoich': 1,
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    },
    'O': {
        'stoich': 2,
        #'E_d': 20.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=11.5, width=10000.0) #density from PuO2 wikipedia

2. Example ion is Ca-48.

In [ ]:
# Construct ion
A, E_per_A_max = 48, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('Ca', energy=energy_max, mass=A)

3. Calculating mean/effective nuclear charge for reference layer and desired layer.

In [ ]:
Z_UO2 = (92+2*8)/3.
Z_PuO2 = (94+2*8)/3.

4. Generate stopping power.

In [ ]:
PuO2 = GenerateStoppingPower(layer, ion, Z=Z_PuO2, Z_ref=Z_UO2)

Creating stopping power for U02, as comparison.

In [ ]:
UO2 = GenerateStoppingPower(layer, ion)

In [ ]:
plt.figure()
plt.plot(UO2.energy_raw, UO2.dEdX_raw, label='UO2')
plt.plot(PuO2.energy_raw, PuO2.dEdX_raw, label='PuO2')
plt.xlabel('Energy (MeV)')
plt.ylabel('Stopping power (MeV/µm)')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(UO2.energy_raw, UO2.GetRange(UO2.energy_raw), label='UO2')
plt.plot(PuO2.energy_raw, PuO2.GetRange(PuO2.energy_raw, from_sr=True), label='PuO2')
plt.xlabel('Energy (MeV)')
plt.ylabel('Range (µm)')
plt.legend()
plt.show()